# Map Data

In this tutorial, we will work with map data.

In [1]:
import rnet as rn

## The MapData Class

The `MapData` class is used to store map data. Map data may be loaded from an OSM file into a `MapData` object using the `from_osm()` class method.

In [2]:
mapdata = rn.MapData.from_osm("samples/ichihara.osm")

The `dump()` method provides information about the data that has been loaded.

In [6]:
mapdata.dump()

name: ichihara.osm
crs: EPSG:4326
vertex_count: 414,966
link_count: 437,627


Suppose that we require only the roads with the `'trunk'` tag. The `include` keyword may be specified to keep only the features with the desired tag.

In [8]:
mapdata = rn.MapData.from_osm("samples/ichihara.osm", include=['trunk'])
mapdata.dump()

name: ichihara.osm
crs: EPSG:4326
vertex_count: 4,926
link_count: 4,938


The map data are stored in data frames in the `vertices` and `links` attributes.

In [9]:
print(mapdata.vertices, mapdata.links, sep='\n')

               x          y
0     139.941200  35.386776
1     139.941458  35.386933
2     139.941690  35.387097
3     139.941835  35.387203
4     139.942727  35.387880
...          ...        ...
4921  140.263174  35.225245
4922  140.263284  35.225194
4923  140.263915  35.286284
4924  140.265576  35.285880
4925  140.265865  35.285801

[4926 rows x 2 columns]
             tag
i    j          
2979 3010  trunk
3010 3016  trunk
3016 3025  trunk
3025 3029  trunk
3029 3033  trunk
...          ...
3465 3470  trunk
3470 3471  trunk
3471 3473  trunk
3436 3440  trunk
3440 3441  trunk

[4938 rows x 1 columns]


## Elevation Data

Elevation data is loaded from a TIF file into an `ElevationData` object using the `from_tif()` method.

In [2]:
elvdata = rn.ElevationData.from_tif("samples/n35_e139_1arc_v3.tif")

The `dump()` method of the `ElevationData` class provides information about the data that has been loaded.

In [3]:
elvdata.dump()

name: n35_e139_1arc_v3.tif
crs: EPSG:4326
point_count: 12,967,201
xmin: 138.9998611111111
xmax: 139.9998611110761
ymin: 35.00013888889832
ymax: 36.000138888888884
zmin: -58
zmax: 1,731


Now, the elevation at a single point may be computed using the `get_elev()` method.

In [7]:
elvdata.get_elev(139.5,35.5)

61.54879664784125

To load data from multiple source files, load the data into an `ElevationDataContainer` object.

In [8]:
container = rn.ElevationDataContainer()
container.add("samples/n35_e139_1arc_v3.tif")
container.add("samples/n35_e140_1arc_v3.tif")

Then, use the `out()` method of the `ElevationDataContainer` class to create an `ElevationData` instance.

In [9]:
combined = container.out()
combined.dump()

name: Concatenated
crs: EPSG:4326
point_count: 25,934,402
xmin: 138.9998611111111
xmax: 140.9998611110761
ymin: 35.00013888889832
ymax: 36.000138888888884
zmin: -84
zmax: 1,731


The elevation at multiple points may be computed using the `get_elevs()` method.

In [10]:
import numpy as np
import time
N = 10_000
xpoints = np.random.uniform(combined.xmin,combined.xmax,N)
ypoints = np.random.uniform(combined.ymin,combined.ymax,N)
start_time = time.perf_counter()
z = combined.get_elevs(np.column_stack([xpoints,ypoints]))
print(np.column_stack([xpoints,ypoints,z]),
      f'That took {time.perf_counter()-start_time:.4f} secs.',
      sep='\n')

[[140.59078018  35.73183616   4.404668  ]
 [139.25013466  35.35289015 112.97425693]
 [139.36435824  35.84317802  78.18305063]
 ...
 [139.03842846  35.90303239 997.40871456]
 [139.64383235  35.81259033   6.6064601 ]
 [140.12523618  35.02066634   0.        ]]
That took 1.6228 secs.
